In [14]:
from caveclient import CAVEclient
from nglui import parser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

client = CAVEclient('minnie65_public')

In [12]:
url = 'https://neuroglancer.neuvue.io/?json_url=https://global.daf-apis.com/nglstate/api/v1/5298478177583104'

# This splits the url string at every "/" and then takes the last one (the number part) and turns it into an integer.
state_id = int(url.split('/')[-1])

# Download the state
state = client.state.get_state_json(state_id)

Get the annotation dataframes

In [29]:
anno_df = parser.annotation_dataframe(state)

anno_df.head()

,layer,anno_type,point,pointB,linked_segmentation,tags,group_id,description
0,synapses,point,"[159850, 244135, 24485]",NaN,"[864691135494808976, 864691135412658258]",[],None,None
1,synapses,point,"[207682, 218096, 23052]",NaN,"[864691135494808976, 864691135809943756]",[1],None,None
2,synapses,point,"[158738, 245466, 24608]",NaN,"[864691135494808976, 864691135412734546]",[],None,None
3,synapses,point,"[185052, 221602, 23773]",NaN,"[864691135494808976, 864691135608549060]",[1],None,None
4,synapses,point,"[162868, 242334, 24122]",NaN,"[864691135494808976, 864691136349347298]",[],None,None


In [30]:
# Compute the mapping from tag numbers to strings
tag_dictionary = parser.tag_dictionary(state, layer_name='synapses')
tag_dictionary

{1: 'mushroom', 2: 'stubby', 3: 'thin', 4: 'idk'}

In [31]:
# this adds one column per tag category with a true/false value in each row

for k, v in tag_dictionary.items():
    new_col = [np.isin(k, tag_list) for tag_list in anno_df.tags]
    anno_df[v] = new_col

In [33]:
anno_df.head()

,layer,anno_type,point,pointB,linked_segmentation,tags,group_id,description,mushroom,stubby,thin,idk
0,synapses,point,"[159850, 244135, 24485]",NaN,"[864691135494808976, 864691135412658258]",[],None,None,False,False,False,False
1,synapses,point,"[207682, 218096, 23052]",NaN,"[864691135494808976, 864691135809943756]",[1],None,None,True,False,False,False
2,synapses,point,"[158738, 245466, 24608]",NaN,"[864691135494808976, 864691135412734546]",[],None,None,False,False,False,False
3,synapses,point,"[185052, 221602, 23773]",NaN,"[864691135494808976, 864691135608549060]",[1],None,None,True,False,False,False
4,synapses,point,"[162868, 242334, 24122]",NaN,"[864691135494808976, 864691136349347298]",[],None,None,False,False,False,False
